### 1. Load Packages

In [1]:
import openai
import pandas as pd
import re
import json

### 2. Load Tokens

In [2]:
openai.api_key = "To be filled"

### 3. Load Data or Text

In [3]:
df1 = pd.read_csv('processed_60900_61899.csv')
df2 = pd.read_csv('processed_177000_177999.csv')

In [4]:
df1

,Unnamed: 0,GLOBAL_ID,LOCAL_ID,processed_txt,date,_date_,WoY
0,60900,60900,6555,we already have congestion pricing for taxis &...,2021-07-09 00:02:34+00:00,2021-07-09,2021-27
1,60901,60901,6556,an the best cannot be that much i am sorry htt...,2021-07-08 23:44:03+00:00,2021-07-08,2021-27
2,60902,60902,6557,handicap uber's is getting out of hand,2021-07-08 23:40:10+00:00,2021-07-08,2021-27
3,60903,60903,6558,bitchh i would calmly get in a cab laugh ill b...,2021-07-08 23:16:21+00:00,2021-07-08,2021-27
4,60904,60904,6559,take a cab. you are getting hepatitis walking ...,2021-07-08 23:15:04+00:00,2021-07-08,2021-27
...,...,...,...,...,...,...,...
995,61895,61895,7550,so...i confess ...i walked to target &amp; bou...,2021-06-17 23:27:09+00:00,2021-06-17,2021-24
996,61896,61896,7551,"uber, ferry, plane, train today",2021-06-17 23:00:01+00:00,2021-06-17,2021-24
997,61897,61897,7552,"hi, i order some food about a hour ago and i h...",2021-06-17 22:08:29+00:00,2021-06-17,2021-24
998,61898,61898,7553,i was there. it was an uber. none of the other...,2021-06-17 21:54:38+00:00,2021-06-17,2021-24


In [5]:
df2

,Unnamed: 0,GLOBAL_ID,LOCAL_ID,processed_txt,date,_date_,WoY
0,177000,177000,7102,please kick japan out of my country they are s...,2021-11-26 13:17:13+00:00,2021-11-26,2021-47
1,177001,177001,7103,once i learned about correctional exercises an...,2021-11-26 12:41:27+00:00,2021-11-26,2021-47
2,177002,177002,7104,empty train cars &gt;&gt;,2021-11-26 12:22:52+00:00,2021-11-26,2021-47
3,177003,177003,7105,that emerald parking lot is about 1 mile walk ...,2021-11-26 12:14:05+00:00,2021-11-26,2021-47
4,177004,177004,7106,"that is going to take time, i need car by end ...",2021-11-26 11:48:54+00:00,2021-11-26,2021-47
...,...,...,...,...,...,...,...
995,177995,177995,8097,how about getting all trash off the sidewalk? ...,2021-06-03 02:03:54+00:00,2021-06-03,2021-22
996,177996,177996,8098,get trash off the sidewalk. trash corals in th...,2021-06-03 02:00:17+00:00,2021-06-03,2021-22
997,177997,177997,8099,people of philadelphia find trae young's hotel...,2021-06-03 01:55:00+00:00,2021-06-03,2021-22
998,177998,177998,8100,ring rain coming back for more and do not you ...,2021-06-03 01:40:50+00:00,2021-06-03,2021-22


### 4.Chain of thought

In [6]:
def generate_chain_of_thought(tweet):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages = [
            {
                "role": "system",
                "content": "You are an AI specifically trained to extract travel experiences from tweets, focusing on the mode of transportation, the sentiment towards it, and the reasons behind the sentiment. It's crucial to maintain consistency in your analysis, avoiding assumptions not supported by the tweet's content (overfitting), yet also ensuring to provide as much detail as possible without defaulting to 'na' unless absolutely necessary (underfitting). Your responses should be in lowercase and adhere to the specified categories."
            },
            {
                "role": "user",
                "content": f"Analyze the tweet: '{tweet.lower()}'. Your goal is to clearly identify the travel experience it describes, focusing on these aspects:\n\n"
                   "1. **Mode of Transportation:** Identify the primary mode of transportation mentioned (choices: 'car', 'subway', 'bus', 'bike', 'taxi','uber','lyft' or 'na' if none is explicitly mentioned or can be inferred from the context). Use context clues to make an educated guess where direct mentions are lacking. If multiple modes are mentioned, determine which is most emphasized by the author as their mode of choice.\n"
                   "2. **Sentiment Analysis:** Determine the sentiment expressed towards this transportation mode ('positive', 'negative', 'neutral'). If the tweet's language is ambiguous or the sentiment is not directly expressed, use contextual evidence to infer the sentiment or resort to 'na' if such inference is not possible.\n"
                   "3. **Reasoning Behind Sentiment:** Extract any reasons or factors mentioned that influence the sentiment. If specific reasons are unclear or not stated, summarize the contextual hints or impressions that might hint at underlying reasons. If no such context exists, use 'na'.\n\n"
                   "Conclude with a concise, structured summary (keep this format for all your response) in JSON format, like this: "
                   "{\"travel_mode\": \"<inferred mode>\", \"sentiment\": \"<inferred sentiment>\", \"reasons\": \"<inferred reasons>\"}. Remember, consistency and adherence to the evidence presented in the tweet are key."
            }
        ],
        temperature=0.7,  # A slightly higher temperature for creativity in judgment
        max_tokens=250
    )
    content = response.choices[0].message.content if response.choices else "No response"
    return content
    

generate some-grade input(train)-very important idea-in-context learning-LLM future reasoners-
Rethinking the Role of Demonstrations: What Makes In-Context Learning Work? - in context

In [7]:
def process_tweet_and_generate_output(df):
    processed_data = []
    
    for index, row in df.iterrows():
        analysis_result_json = generate_chain_of_thought(row['processed_txt'])
        
        try:
            # Attempt to parse the JSON string
            analysis_result = json.loads(analysis_result_json)
        except json.JSONDecodeError:
            # If parsing fails, use default values
            analysis_result = {"travel_mode": "na", "sentiment": "neutral", "reasons": "Parsing error or invalid format in response."}
        
        # Extract and store the relevant information along with the GLOBAL_ID and original tweet
        processed_row = {
            "GLOBAL_ID": row['GLOBAL_ID'],
            "Tweet": row['processed_txt'],
            "Travel Mode": analysis_result.get('travel_mode', 'NA'),
            "Sentiment": analysis_result.get('sentiment', 'NA'),
            "Reasons": analysis_result.get('reasons', 'NA')
        }
        
        # Append the processed row to the list of processed data
        processed_data.append(processed_row)

    # Convert the processed data into a DataFrame
    return pd.DataFrame(processed_data)

### 5.Outcome

**(1) df1: processed_60900_61899.csv**

In [8]:
# test
process_tweet_and_generate_output(df1.iloc[0:2])

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons
0,60900,we already have congestion pricing for taxis &...,car,positive,congestion pricing for cars and trucks is inte...
1,60901,an the best cannot be that much i am sorry htt...,na,na,na


In [9]:
processed_df1 = process_tweet_and_generate_output(df1)
processed_df1.to_csv('result_df1_edit.csv', index=False)

In [10]:
processed_df1

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons
0,60900,we already have congestion pricing for taxis &...,car,positive,congestion pricing aims to encourage the use o...
1,60901,an the best cannot be that much i am sorry htt...,na,na,The tweet does not provide clear information r...
2,60902,handicap uber's is getting out of hand,uber,negative,The sentiment is negative due to the frustrati...
3,60903,bitchh i would calmly get in a cab laugh ill b...,taxi,positive,The sentiment towards taking a taxi seems posi...
4,60904,take a cab. you are getting hepatitis walking ...,taxi,negative,the author expresses a negative sentiment towa...
...,...,...,...,...,...
995,61895,so...i confess ...i walked to target &amp; bou...,uber,positive,the author expresses excitement and satisfacti...
996,61896,"uber, ferry, plane, train today",mixed,neutral,The tweet mentions using multiple modes of tra...
997,61897,"hi, i order some food about a hour ago and i h...",na,negative,the author is frustrated as their food order h...
998,61898,i was there. it was an uber. none of the other...,uber,neutral,the sentiment towards taking an Uber is neutra...


In [11]:
processed_df1['Travel Mode'].value_counts().get('na', 0)

257

**(2) df2: processed_177000_177999.csv**

In [12]:
processed_df2 = process_tweet_and_generate_output(df2)
processed_df2.to_csv('result_df2_edit.csv', index=False)

In [13]:
processed_df2

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons
0,177000,please kick japan out of my country they are s...,car,negative,The sentiment towards the car transportation m...
1,177001,once i learned about correctional exercises an...,car,neutral,The comparison made in the tweet suggests a ne...
2,177002,empty train cars &gt;&gt;,train,positive,The author expresses a positive sentiment towa...
3,177003,that emerald parking lot is about 1 mile walk ...,car,negative,"The primary mode of transportation is by car, ..."
4,177004,"that is going to take time, i need car by end ...",car,neutral,the tweet indicates a need for a car by the en...
...,...,...,...,...,...
995,177995,how about getting all trash off the sidewalk? ...,car,negative,The sentiment towards the car is negative due ...
996,177996,get trash off the sidewalk. trash corals in th...,car,negative,The author expresses a negative sentiment towa...
997,177997,people of philadelphia find trae young's hotel...,car,negative,The sentiment expressed is negative as the twe...
998,177998,ring rain coming back for more and do not you ...,car,positive,The sentiment towards the car ride seems posit...


In [14]:
processed_df2['Travel Mode'].value_counts().get('na', 0)

48

### 6. Verfication

In [15]:
def verify_llm_output(tweet, travel_mode, sentiment, reasons):
    verification_prompt = f"""
    Based on the following tweet analysis:
    Tweet: {tweet}
    - Travel Mode: {travel_mode}
    - Sentiment: {sentiment}
    - Reasons: {reasons}

    Is this analysis 'correct' or 'incorrect'? Consider the relevance and accuracy of the travel mode, sentiment, and reasons given.
    """
    
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{
            "role": "system",
            "content": "You're a highly intelligent assistant. Please read the following analysis carefully and provide your verdict."
        }, {
            "role": "user",
            "content": verification_prompt
        }],
        temperature=0,  # Use 0 for deterministic output
        max_tokens=50
    )

    # Simplifying the response check
    text_response = response.choices[0].message.content.strip().lower()
    if "correct" in text_response:
        return "correct"
    elif "incorrect" in text_response:
        return "incorrect"
    else:
        return "undetermined"  # Use this if the response isn't clear


**(1) df1: processed_60900_61899.csv**

In [16]:
# test
verify_llm_output(processed_df1.iloc[0]['Tweet'],processed_df1.iloc[0]['Travel Mode'], processed_df1.iloc[0]['Sentiment'], processed_df1.iloc[0]['Reasons'])

'correct'

In [17]:
processed_df1['Verification Result'] = processed_df1.apply(
    lambda row: verify_llm_output(row['Tweet'], row['Travel Mode'], row['Sentiment'], row['Reasons']), axis=1)

In [18]:
processed_df1

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons,Verification Result
0,60900,we already have congestion pricing for taxis &...,car,positive,congestion pricing aims to encourage the use o...,correct
1,60901,an the best cannot be that much i am sorry htt...,na,na,The tweet does not provide clear information r...,correct
2,60902,handicap uber's is getting out of hand,uber,negative,The sentiment is negative due to the frustrati...,correct
3,60903,bitchh i would calmly get in a cab laugh ill b...,taxi,positive,The sentiment towards taking a taxi seems posi...,correct
4,60904,take a cab. you are getting hepatitis walking ...,taxi,negative,the author expresses a negative sentiment towa...,correct
...,...,...,...,...,...,...
995,61895,so...i confess ...i walked to target &amp; bou...,uber,positive,the author expresses excitement and satisfacti...,correct
996,61896,"uber, ferry, plane, train today",mixed,neutral,The tweet mentions using multiple modes of tra...,correct
997,61897,"hi, i order some food about a hour ago and i h...",na,negative,the author is frustrated as their food order h...,correct
998,61898,i was there. it was an uber. none of the other...,uber,neutral,the sentiment towards taking an Uber is neutra...,correct


In [19]:
processed_df1.to_csv('Week 7 result_60900_61899((after self-verification).csv', index=False)

In [20]:
processed_df1['Verification Result'].value_counts().get('correct', 0)

998

**(2) df2: processed_177000_177999.csv**

In [21]:
processed_df2['Verification Result'] = processed_df2.apply(
    lambda row: verify_llm_output(row['Tweet'], row['Travel Mode'], row['Sentiment'], row['Reasons']), axis=1)

In [22]:
processed_df2

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons,Verification Result
0,177000,please kick japan out of my country they are s...,car,negative,The sentiment towards the car transportation m...,correct
1,177001,once i learned about correctional exercises an...,car,neutral,The comparison made in the tweet suggests a ne...,correct
2,177002,empty train cars &gt;&gt;,train,positive,The author expresses a positive sentiment towa...,correct
3,177003,that emerald parking lot is about 1 mile walk ...,car,negative,"The primary mode of transportation is by car, ...",correct
4,177004,"that is going to take time, i need car by end ...",car,neutral,the tweet indicates a need for a car by the en...,correct
...,...,...,...,...,...,...
995,177995,how about getting all trash off the sidewalk? ...,car,negative,The sentiment towards the car is negative due ...,correct
996,177996,get trash off the sidewalk. trash corals in th...,car,negative,The author expresses a negative sentiment towa...,correct
997,177997,people of philadelphia find trae young's hotel...,car,negative,The sentiment expressed is negative as the twe...,correct
998,177998,ring rain coming back for more and do not you ...,car,positive,The sentiment towards the car ride seems posit...,correct


In [23]:
processed_df2.to_csv('Week 7 result_177000_177999(after self-verification).csv', index=False)

In [24]:
processed_df2['Verification Result'].value_counts().get('correct', 0)

1000

## 6. Output Evaluation

In [5]:
df1_res=pd.read_csv('Week 7 result_60900_61899((after self-verification).csv')
df1_res

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons,Verification Result
0,60900,we already have congestion pricing for taxis &...,car,positive,congestion pricing aims to encourage the use o...,correct
1,60901,an the best cannot be that much i am sorry htt...,na,na,The tweet does not provide clear information r...,correct
2,60902,handicap uber's is getting out of hand,uber,negative,The sentiment is negative due to the frustrati...,correct
3,60903,bitchh i would calmly get in a cab laugh ill b...,taxi,positive,The sentiment towards taking a taxi seems posi...,correct
4,60904,take a cab. you are getting hepatitis walking ...,taxi,negative,the author expresses a negative sentiment towa...,correct
...,...,...,...,...,...,...
995,61895,so...i confess ...i walked to target &amp; bou...,uber,positive,the author expresses excitement and satisfacti...,correct
996,61896,"uber, ferry, plane, train today",mixed,neutral,The tweet mentions using multiple modes of tra...,correct
997,61897,"hi, i order some food about a hour ago and i h...",na,negative,the author is frustrated as their food order h...,correct
998,61898,i was there. it was an uber. none of the other...,uber,neutral,the sentiment towards taking an Uber is neutra...,correct


In [7]:
def analyze_data(df):
    # Analyze 'Travel Mode'
    unique_travel_modes = df['Travel Mode'].unique()
    print("Distinct outcomes for Travel Mode:", unique_travel_modes)

    number_of_distinct_travel_modes = df['Travel Mode'].nunique()
    print("Number of distinct outcomes for Travel Mode:", number_of_distinct_travel_modes)

    travel_mode_counts = df['Travel Mode'].value_counts()
    print("Counts for each distinct outcome in Travel Mode:", travel_mode_counts)

    # Analyze 'Sentiment'
    unique_sentiments = df['Sentiment'].unique()
    print("Distinct outcomes for Sentiment:", unique_sentiments)

    number_of_distinct_sentiments = df['Sentiment'].nunique()
    print("Number of distinct outcomes for Sentiment:", number_of_distinct_sentiments)

    sentiment_counts = df['Sentiment'].value_counts()
    print("Counts for each distinct outcome in Sentiment:", sentiment_counts)

    # Calculate various rates
    extraction_rate = len(df[~df['Travel Mode'].isin(['na', 'NA'])]) / len(df)
    satisfaction_rate = len(df[df['Sentiment'] == 'positive']) / len(df)
    self_verification_rate = len(df[df['Verification Result'] == 'correct']) / len(df)
    
    if 'Cross Verification Result' in df.columns:
        cross_verification_rate = len(df[df['Cross Verification Result'] == 'correct']) / len(df)
    else:
        cross_verification_rate = None
    
    return {
        'Extraction Rate': extraction_rate,
        'Satisfaction Rate': satisfaction_rate,
        'Self Verification Rate': self_verification_rate
    }


In [8]:
analyze_data(df1_res)

Distinct outcomes for Travel Mode: ['car' 'na' 'uber' 'taxi' 'walking' 'lyft' 'subway' 'bus' 'bike'
 'food delivery' 'flight' nan 'food delivery service' 'plane'
 'ride-sharing (uber/lyft)' 'ride' 'ride (car)'
 'public transportation (subway)' 'flights' 'delivery' 'courier'
 'delivery service' 'pool' 'public transit' 'public transportation'
 'train' 'ambulance' 'ride service (uber/lyft)' 'public trans'
 'water taxi' 'cab' 'boat taxi' 'foot' 'dollar van' 'scooter' 'mixed']
Number of distinct outcomes for Travel Mode: 35
Counts for each distinct outcome in Travel Mode: car                               300
na                                257
taxi                              209
uber                               60
bike                               42
lyft                               42
bus                                16
walking                            13
subway                              9
plane                               8
train                               5
food del

{'Extraction Rate': 0.743,
 'Satisfaction Rate': 0.249,
 'Self Verification Rate': 0.998}